<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/10-Optimizacion-Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Escuela de Ciencias Físicas y Matemática</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optmización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Andrés Merino &bull; 2025-04
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

A lo largo de este taller, aplicaremos optimización de hiperparámetros en un modelo que elijas.

Los paquetes necesarios son:

In [22]:
# Paquetes necesarios
import pandas as pd  # Manejo de datos
import matplotlib.pyplot as plt  # Visualización

from sklearn.model_selection import train_test_split # División de datos
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métrica de evaluación

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold  # Búsqueda de hiperparámetros
from sklearn.tree import DecisionTreeClassifier, plot_tree

from scipy.stats import randint

import joblib  # para guardar y cargar modelos

---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilzar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

In [3]:
# Cargar datos
df = pd.read_csv("https://raw.githubusercontent.com/danirodriguez4657/datos_diabetes/refs/heads/main/diabetes_limpio_reducidos.csv")

# Mostrar los primeros datos
print("Primeras filas del dataset:")
print(df.head())

# Descripción estadística de los datos
print("\nDescripción estadística:")
print(df.describe())

# Separar las variables predictoras
X = df.drop("Outcome", axis=1)  # Variables predictoras
y = df["Outcome"]               # Variable objetivo

#Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

Primeras filas del dataset:
   Glucose   BMI  Age  BloodPressure  DiabetesPedigreeFunction  Outcome
0      148  33.6   50             72                     0.627        1
1       85  26.6   31             66                     0.351        0
2      183  23.3   32             64                     0.672        1
3       89  28.1   21             66                     0.167        0
4      137  43.1   33             40                     2.288        1

Descripción estadística:
          Glucose         BMI         Age  BloodPressure  \
count  768.000000  768.000000  768.000000     768.000000   
mean   120.894531   31.992578   33.240885      69.105469   
std     31.972618    7.884160   11.760232      19.355807   
min      0.000000    0.000000   21.000000       0.000000   
25%     99.000000   27.300000   24.000000      62.000000   
50%    117.000000   32.000000   29.000000      72.000000   
75%    140.250000   36.600000   41.000000      80.000000   
max    199.000000   67.100000   81


### <font color='264CC7'> Modelo </font>


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Selecciona el mejor modelo de las clases anteriores.
<ul>
  <li>Muestra los hiperparámetros del modelo.</li>
  <li>Consulta qué significan al menos 4 hiperparámetros.</li>
  <li>Selecciona los hiperparámetros que deseas optimizar, al menos 3.</li>
</ul>
</div>

In [12]:
# Crear y entrenar un arbol con ganancia de información
modelo_base = DecisionTreeClassifier(random_state=42)
modelo_base.fit(X_train, y_train)

# Imprimir los parametros del modelo
print(modelo_base.get_params())

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': 42, 'splitter': 'best'}


| Hiperparámetro      | Significado                                                                                 |
| ------------------- | ------------------------------------------------------------------------------------------- |
| `criterion`         | Criterio para medir la calidad de una división. Puede ser `'gini'` o `'entropy'`.           |
| `max_depth`         | Profundidad máxima del árbol. Limita cuánto puede crecer. Previene sobreajuste.             |
| `min_samples_split` | Número mínimo de muestras necesarias para dividir un nodo interno.                          |
| `min_samples_leaf`  | Número mínimo de muestras que debe tener una hoja. Controla el tamaño mínimo de los grupos. |


### <font color='264CC7'> Optimización por GridSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica GridSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 3 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [13]:
# Parámetros para optimizar (con al menos 3 valores cada uno)
parametros = {
    'max_depth': [None, 3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

In [14]:
# Validación cruzada KFold con 5 splits
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

In [15]:
# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=modelo_base,
                           param_grid=parametros,
                           cv=k_fold,
                           scoring='accuracy',
                           n_jobs=-1)

# Entrenar GridSearch con tus datos (X_train, y_train)
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros y score
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor score (accuracy):", grid_search.best_score_)

Mejores hiperparámetros: {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 2}
Mejor score (accuracy): 0.7442889510862322


- Grid Search encontró un modelo más simple con profundidad 3, con criterio gini y parámetros que permiten un árbol menos profundo, con un accuracy mas bajo pero por poco (≈0.74).

### <font color='264CC7'> Optimización por RandomSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica RandomSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Usa RandomSearchCV con 25 iteraciones.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [20]:
# Espacio de búsqueda para los hiperparámetros con al menos 5 opciones
param_dist = {
    'max_depth': [None, 3, 5, 7, 10, 15],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'criterion': ['gini', 'entropy'],
    'max_features': [None, 'sqrt', 'log2']  # quitar 'auto'
}


In [21]:
# Validación cruzada con 5 splits
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

# RandomizedSearchCV con 25 iteraciones
random_search = RandomizedSearchCV(estimator=modelo_base,
                                   param_distributions=param_dist,
                                   n_iter=25,
                                   cv=k_fold,
                                   scoring='accuracy',
                                   random_state=42,
                                   n_jobs=-1)

# Entrenar el RandomizedSearch (usa tus datos reales en lugar de X_train, y_train)
random_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros y score
print("Mejores hiperparámetros:", random_search.best_params_)
print("Mejor score (accuracy):", random_search.best_score_)

Mejores hiperparámetros: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 18, 'min_samples_split': 10}
Mejor score (accuracy): 0.7638278022124483


- Random Search encontró un modelo más complejo con profundidad ilimitada con restricciones más fuertes para evitar sobreajuste (min_samples_leaf=18, min_samples_split=10), usando el criterio entropy y limitando características con sqrt. Esto le da un mejor balance por lo que el accuracy resultó mayor (≈0.76).

### <font color='264CC7'> Guardado de modelo </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Con los parámetros óptimos que mejor resultado dieron, reentrena el modelo, muestra su score y guárdalo.
</div>

In [23]:
# Supongamos que X_train, X_test, y_train, y_test ya están definidos

# Definir el modelo con los mejores hiperparámetros encontrados en Random Search
modelo_final = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=None,
    max_features='sqrt',
    min_samples_leaf=18,
    min_samples_split=10,
    random_state=42
)

# Entrenas el modelo con todo el conjunto de entrenamiento
modelo_final.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
score = modelo_final.score(X_test, y_test)
print(f"Accuracy del modelo final en test: {score:.4f}")


Accuracy del modelo final en test: 0.7208


In [24]:
# Guardar el modelo
joblib.dump(modelo_final, 'modelo_arbol_decision_diabetes.joblib')

['modelo_arbol_decision_diabetes.joblib']

### <font color='264CC7'> Publicación </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Coloca el este cuaderno y el modelo en tu repositorio de GitHub. Agrega una licencia MIT y un README.md donde se explique el contenido del repositorio, los datos utilizados y los resultados obtenidos.
</div>